# 릿지 회귀모델

---

## 1. 분석 데이터 준비

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [4]:
data2 = pd.read_csv('data/house_price.csv')
data2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [5]:
# 특성 데이터 분류
X=data2[data2.columns[1:5]]
y=data2[["house_value"]]

In [6]:
# 훈련, 테스트 데잏터 분류
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=42)

In [8]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)
X_scaled_train=scaler.transform(X_train)
X_scaled_test=scaler.transform(X_test)

## 2. 기본 모델 적용 (릿지 모델)

In [9]:
from sklearn.linear_model import Ridge

In [10]:
model=Ridge()
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.5455487773718164

In [11]:
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.5626954941458684

In [12]:
# RMSE (Root Mean Squared Error)
import numpy as np
from sklearn.metrics import mean_squared_error 
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련   데이터 RMSE:", np.sqrt(MSE_train))
print("테스트 데이터 RMSE:", np.sqrt(MSE_test))

훈련   데이터 RMSE: 64342.018619526265
테스트 데이터 RMSE: 63219.99395904853


## 3.Grid Search

In [14]:
from sklearn.model_selection import GridSearchCV

In [13]:
param_grid = {'alpha' : [1e-4, 1e-3, 1e-2, 0.1, 0.5, 1.0, 5.0, 10.0]}

In [15]:
grid_search=GridSearchCV(Ridge(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 5.0,
                                   10.0]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [17]:
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter: {'alpha': 0.1}
Best Score: 0.5452
TestSet Score: 0.5627


## 4.Random Search

In [19]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [21]:
param_distribs = {'alpha': randint(low=0.0001, high=100)}

In [22]:
random_search=RandomizedSearchCV(Ridge(), 
                                 param_distributions=param_distribs, n_iter=100, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001),
                   iid='deprecated', n_iter=100, n_jobs=None,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa27f249ac0>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [23]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter: {'alpha': 0}
Best Score: 0.5452
TestSet Score: 0.5627
